# [1단계] 환경 설정 및 데이터 다운로드

## 1-1. 필요한 라이브러리 설치

In [1]:
# gdown: 구글 드라이브에 있는 파일을 ID를 이용해 쉽게 다운로드하게 해주는 라이브러리
# datasets, pandas, huggingface_hub: 이후 작업에 필요하므로 함께 설치
!pip install -qU gdown datasets "pandas==2.2.2" huggingface_hub pdfplumber

## 1-2. 라이브러리 임포트

In [2]:
import gdown
import pandas as pd
from datasets import load_dataset
print("✅ 라이브러리 준비 완료!")

✅ 라이브러리 준비 완료!


## 1-3. gdown을 사용하여 구글 드라이브에서 CSV 파일 다운로드

In [3]:
# file share address: https://drive.google.com/file/d/1fUcRKDUM8aXrK5oP4KvKl2HI07PYsWmX/view?usp=sharing
file_id = "1fUcRKDUM8aXrK5oP4KvKl2HI07PYsWmX"
output_filename = "KB_sLLM_QA_Dataset.csv"

# gdown.download 함수를 사용해 파일을 다운로드한다.
gdown.download(id=file_id, output=output_filename, quiet=False)

print(f"\n✅ '{output_filename}' 파일 다운로드 완료!")

Downloading...
From: https://drive.google.com/uc?id=1fUcRKDUM8aXrK5oP4KvKl2HI07PYsWmX
To: /content/KB_sLLM_QA_Dataset.csv
100%|██████████| 478k/478k [00:00<00:00, 75.4MB/s]


✅ 'KB_sLLM_QA_Dataset.csv' 파일 다운로드 완료!


## 1-4. 다운로드된 CSV 파일 로드 및 확인

In [4]:
# pandas를 사용해 다운로드한 파일을 읽고, 내용이 잘 들어있는지 확인
df = pd.read_csv(output_filename)
print(f"\n--- 로드된 데이터 정보 ---")
print(f"총 {len(df)}개의 Q&A 데이터를 로드했습니다.")
print("\n--- 데이터 샘플 (상위 3개) ---")
print(df.head(3))


--- 로드된 데이터 정보 ---
총 1157개의 Q&A 데이터를 로드했습니다.

--- 데이터 샘플 (상위 3개) ---
                                            question  \
0                            CD수익률의 정의와 그 용도는 무엇인가요?   
1        CD수익률 산출 과정에서 오류가 발생했을 경우, 협회는 어떤 절차를 따르나요?   
2  비상사태 발생 시 CD수익률 산출업무 중단에 따른 금융계약의 대체금리는 어떻게 결정...   

                                            answer_B         source_pdf  
0  CD수익률이란 신용평가회사로부터 AAA이상의 신용등급을 받은 시중은행이 발행한 만기...  pdf_0_label_5.pdf  
1  협회는 오류가 발견된 경우, 오류의 발생 원인에 따라 다음과 같은 절차를 따릅니다....  pdf_0_label_5.pdf  
2  비상사태 발생으로 CD수익률 산출업무가 중단되는 경우, 해당 중단기간 동안 사용기관...  pdf_0_label_5.pdf  


# [2단계] 원본 데이터셋 로드 및 이름 매핑 준비

## 2-1. Hugging Face 로그인

In [5]:
from huggingface_hub import login
from google.colab import userdata

try:
    hf_token = userdata.get('HF_TOKEN')
    login(token=hf_token)
    print("✅ Hugging Face 로그인 성공!")
except Exception as e:
    print(f"🚨 Hugging Face 로그인 실패: {e}")

✅ Hugging Face 로그인 성공!


## 2-2. 실제 파일 이름 정보를 담고 있는 원본 데이터셋 로드

In [6]:
try:
    source_dataset = load_dataset("sumilee/SKN14-Final-3Team-Data2", split="train")
    print(f"\n✅ 원본 파일 이름 출처 데이터셋 로드 완료: {len(source_dataset)}건")
except Exception as e:
    print(f"🚨 원본 데이터셋 로드 실패: {e}")

Resolving data files:   0%|          | 0/233 [00:00<?, ?it/s]


✅ 원본 파일 이름 출처 데이터셋 로드 완료: 233건


# [3단계] 실제 PDF 파일 이름으로 교체

## 3-1. 원본 데이터셋에서 실제 파일 이름 목록 추출

In [16]:
# [디버깅] pdfplumber.PDF 객체 내부 속성 확인

# source_dataset에서 첫 번째 PDF 객체 하나만 가져온다.
first_pdf_object = source_dataset['pdf'][0]

# dir() 함수는 해당 객체가 가지고 있는 모든 속성(변수)과 메서드(함수)의 목록을 보여준다.
# 이걸 보면 우리가 사용할 수 있는 정보가 무엇인지 정확히 알 수 있어.
print("--- 첫 번째 PDF 객체의 내부 속성 목록 ---")
print(dir(first_pdf_object))

# 객체 자체를 출력해보면 더 유용한 정보가 나올 수도 있다.
print("\n--- 객체 자체 출력 ---")
print(first_pdf_object)

--- 첫 번째 PDF 객체의 내부 속성 목록 ---
['__annotations__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__enter__', '__eq__', '__exit__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'annots', 'cached_properties', 'chars', 'close', 'curve_edges', 'curves', 'doc', 'edges', 'flush_cache', 'horizontal_edges', 'hyperlinks', 'images', 'laparams', 'lines', 'metadata', 'objects', 'open', 'pages', 'pages_to_parse', 'password', 'path', 'raise_unicode_errors', 'rect_edges', 'rects', 'rsrcmgr', 'stream', 'stream_is_external', 'structure_tree', 'textboxhorizontals', 'textboxverticals', 'textlinehorizontals', 'textlineverticals', 'to_csv', 'to_dict', 'to_json', 'unicode_norm', 'vertical_edges']

--- 객체 자체 출력 ---


In [17]:
# [3단계] 실제 PDF 파일 이름으로 교체 (path 속성 사용)

## 3-1. 원본 데이터셋에서 실제 파일 이름 목록 추출
import os

# 디버깅을 통해 알아낸 정확한 속성 '.path'를 사용한다.
real_filenames = [os.path.basename(item.path) for item in source_dataset['pdf']]

print("--- 실제 파일 이름 샘플 (상위 5개) ---")
print(real_filenames[:5])

--- 실제 파일 이름 샘플 (상위 5개) ---
['KB_CD수익률표준설명서(가계).pdf', 'KB_개인정보보호_정책.pdf', 'KB_공정한_채권회수를_위한_정책.pdf', 'KB_금리인하요구권.pdf', 'KB_금융소비자보호_상품개발준칙.pdf']


## 3-2. 기존 DataFrame에 실제 파일 이름 매핑

In [18]:
# 1. 'source_pdf' 컬럼에서 정규표현식을 사용해 'pdf_숫자_' 부분의 숫자만 추출한다.
#    예: 'pdf_12_label_5.pdf' -> 12
#    .str.extract()는 문자열에서 특정 패턴을 찾아주는 기능이야.
#    r'pdf_(\d+)_'는 "pdf_"로 시작하고 "_"로 끝나는 패턴 사이의 숫자(\d+)를 찾으라는 의미.
df['pdf_index'] = df['source_pdf'].str.extract(r'pdf_(\d+)_').astype(int)

# 2. 추출한 인덱스를 사용해 real_filenames 리스트에서 실제 파일 이름을 찾아 새 컬럼에 추가한다.
#    .apply()는 'pdf_index' 컬럼의 각 숫자(idx)마다 지정된 함수를 실행시켜줘.
#    lambda idx: real_filenames[idx]는 각 숫자(idx)를 받아서 real_filenames 리스트의 idx번째 항목을 반환하는 간단한 함수야.
df['filename'] = df['pdf_index'].apply(lambda idx: real_filenames[idx])

# 3. 불필요해진 기존 컬럼들을 정리하고, 새 컬럼의 이름을 최종 목표에 맞게 변경한다.
#    df[['col1', 'col2', ...]]는 원하는 컬럼만 선택하는 방법이야.
df_final = df[['question', 'answer_B', 'filename']].copy()
#    .rename()으로 'filename' 컬럼의 이름을 'source_pdf'로 바꿔준다.
df_final.rename(columns={'filename': 'source_pdf'}, inplace=True)

print("\n\n--- 파일 이름 변경 후 최종 데이터 샘플 ---")
print(df_final.head())



--- 파일 이름 변경 후 최종 데이터 샘플 ---
                                            question  \
0                            CD수익률의 정의와 그 용도는 무엇인가요?   
1        CD수익률 산출 과정에서 오류가 발생했을 경우, 협회는 어떤 절차를 따르나요?   
2  비상사태 발생 시 CD수익률 산출업무 중단에 따른 금융계약의 대체금리는 어떻게 결정...   
3  KB금융그룹의 개인정보보호 정책에서 관리적, 기술적, 물리적 조치에 대해 구체적으로...   
4  KB국민은행의 채권회수 정책에서 채무자의 인간다운 삶을 보호하기 위해 어떤 원칙을 ...   

                                            answer_B              source_pdf  
0  CD수익률이란 신용평가회사로부터 AAA이상의 신용등급을 받은 시중은행이 발행한 만기...   KB_CD수익률표준설명서(가계).pdf  
1  협회는 오류가 발견된 경우, 오류의 발생 원인에 따라 다음과 같은 절차를 따릅니다....   KB_CD수익률표준설명서(가계).pdf  
2  비상사태 발생으로 CD수익률 산출업무가 중단되는 경우, 해당 중단기간 동안 사용기관...   KB_CD수익률표준설명서(가계).pdf  
3  KB금융그룹은 개인정보보호를 위해 관리적 조치로 내부관리규정 수립·시행, 정기적 직...        KB_개인정보보호_정책.pdf  
4  KB국민은행은 채권회수를 위해 권리를 남용하거나 불법적인 방법으로 채권을 회수하지 ...  KB_공정한_채권회수를_위한_정책.pdf  


# [4단계] Train / Test 데이터셋 분할 및 업로드

## 4-1. Pandas DataFrame을 Datasets 객체로 변환

In [19]:
from datasets import Dataset, DatasetDict

final_dataset = Dataset.from_pandas(df_final)
print("✅ Pandas DataFrame을 Dataset 객체로 변환 완료!")
print(final_dataset)

✅ Pandas DataFrame을 Dataset 객체로 변환 완료!
Dataset({
    features: ['question', 'answer_B', 'source_pdf'],
    num_rows: 1157
})


## 4-2. 데이터셋 분할

In [20]:
# test_size=150으로 설정하여 평가셋이 최소 150개가 되도록 보장한다.
# random_state (seed)를 고정하여 항상 동일하게 섞이고 분할되도록 한다.
split_dataset_dict = final_dataset.train_test_split(test_size=150, seed=42)

print("\n--- 데이터셋 분할 완료 ---")
print("훈련 데이터셋:", len(split_dataset_dict['train']))
print("테스트 데이터셋:", len(split_dataset_dict['test']))
print(split_dataset_dict)


--- 데이터셋 분할 완료 ---
훈련 데이터셋: 1007
테스트 데이터셋: 150
DatasetDict({
    train: Dataset({
        features: ['question', 'answer_B', 'source_pdf'],
        num_rows: 1007
    })
    test: Dataset({
        features: ['question', 'answer_B', 'source_pdf'],
        num_rows: 150
    })
})


## 4-3. 최종 데이터셋을 Hugging Face Hub에 업로드

In [21]:
try:
    hf_username = userdata.get('HF_USERNAME')
    # 새 데이터셋의 이름을 정해준다.
    repo_name = f"{hf_username}/KB-sLLM-QA-Dataset-Final-Split"

    print(f"\n🚀 최종 데이터셋을 Hugging Face Hub에 업로드합니다...")
    print(f"Repository: {repo_name}")

    # DatasetDict 형태로 업로드하면 'train'과 'test' split이 함께 저장된다.
    split_dataset_dict.push_to_hub(
        repo_id=repo_name,
        private=True, # 비공개로 설정
        commit_message="Split dataset into train/test and update source PDF filenames"
    )

    print("\n🎉 최종 데이터셋 업로드 성공!")
    print("허깅페이스 프로필에서 'train'과 'test' 스플릿을 확인해보세요.")
    print(f"URL: https://huggingface.co/datasets/{repo_name}")

except Exception as e:
    print(f"🚨 데이터셋 업로드 중 오류가 발생했습니다: {e}")


🚀 최종 데이터셋을 Hugging Face Hub에 업로드합니다...
Repository: rucipheryn/KB-sLLM-QA-Dataset-Final-Split


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ? shards/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ? shards/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]


🎉 최종 데이터셋 업로드 성공!
허깅페이스 프로필에서 'train'과 'test' 스플릿을 확인해보세요.
URL: https://huggingface.co/datasets/rucipheryn/KB-sLLM-QA-Dataset-Final-Split
